# import libs

In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers
import keras_cv

Using TensorFlow backend


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  define constants and create dataset


In [2]:
NTRIALS = 300
BATCH_SIZE = 64
EPOCHS = 100

train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

layer_options = ["Conv2D", "Depthwise2D", "Pointwise2D"]
activation_options = ["linear", "relu", "tanh"]
filter_size_options = [3, 4, 5]

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

## define create model function


In [3]:
def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_layer(trial, layer_type, layer_id, max_stride, input_h_size):
    if layer_type == "Conv2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_conv2d_filters", 4, 32, step=4
        )
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_conv2d_fsize", filter_size_options
        )

        stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 2, max_stride)
        layer = layers.Conv2D(
            num_filters, clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Depthwise2D":
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_dwconv2d_ssize", 2, max_stride
        )
        layer = layers.DepthwiseConv2D(
            clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Pointwise2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_ssize", 2, max_stride
        )
        layer = layers.Conv2D(num_filters, 1, stride_size)

    return layer


def create_augmentation_layers(trial, x):
    # augmix_severity = trial.suggest_float("augmix_severity", 0.0, 0.5)
    brightness_factor = trial.suggest_float("brightness_factor", 0, 0.2)
    gaussian_noise_factor = trial.suggest_float("gaussian_noise_factor", 0, 0.2)
    translate_factor = trial.suggest_float("translate_factor", 0, 0.2)

    # x = keras_cv.layers.AugMix(value_range=(0, 1), severity=augmix_severity)(x)
    x = layers.RandomBrightness(brightness_factor, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(translate_factor, translate_factor)(x)
    x = layers.GaussianNoise(gaussian_noise_factor)(x)

    return x


def calibrate_channel(input_shape, s):
    HLS_DENSE_LIMIT = 4096

    x_shape = input_shape[1:]
    h, w, c = x_shape    
    h = h//s
    w = w//s

    new_shape = [h, w, c]
    if np.prod(new_shape) > HLS_DENSE_LIMIT:
        # diminui o numero de canais
        while np.prod(new_shape) > HLS_DENSE_LIMIT:
            c -= 1
            new_shape = [h, w, c]
    else:
        # aumenta o numero de canais
        while np.prod(new_shape) < HLS_DENSE_LIMIT:
            c += 1
            new_shape = [h, w, c]
        c -= 1

    if c <= 0:
        c = 1
        new_shape = [None, h, w, c]        
        h, w, c, s = calibrate_channel(new_shape, s+1)            
        new_shape = [h, w, c]

    return h, w, c, s


def create_model(trial):
    # add the input layer
    x = x_in = layers.Input(shape=(256, 256, 3))

    x = create_augmentation_layers(trial, x)

    # suggent a number of layer
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 3])

    # limits the stride
    max_stride = 16 // num_layers

    # convolutional layers
    for i in range(num_layers):
        layer_selected = trial.suggest_categorical(f"layer_{i}", layer_options)
        layer = create_layer(
            trial, layer_selected, i, max_stride, input_h_size=x.shape[1]
        )
        x = layer(x)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)
        x = layers.Activation(act_selected)(x)

    h, w, c, s = calibrate_channel(x.shape, 1)

    # tinsmith layer
    x = layers.Conv2D(c, 1, strides=s)(x)

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dropout(trial.suggest_float("output_dropout", 0.3, 0.6))(x)
    x = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(x_in, x)

# if testing


```
h, w, c, s = calibrate_channel([None, 84,84,20], 1)

class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        return np.random.randint(min, max)

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):
        print(name)
        return options[np.random.randint(0, len(options) - 1)]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
model.save("saved_models/test/")



```

In [4]:
def create_optimizer(trial):
    lr = trial.suggest_float("lr_inicial", 3e-4, 2e-3, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )



def objective(trial):
    BATCH_SIZE = trial.suggest_int("batchsize", 32, 128, step=32)
    
    # load data
    train_dataset, val_dataset, num_classes = dataset.create_datasets(
        train_images_rgx, valid_images_rgx, BATCH_SIZE
    )

    # Build model and optimizer.
    model = create_model(trial)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf")  # save all model

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="auto",
    )
    
    # load best model
    model.load_weights(saved_best_model_path + "weights/")    

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(val_dataset, verbose=0)
    
    # Return best model validation accuracy.
    return score[1]


def search():
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(),
    )
    study.optimize(objective, n_trials=NTRIALS)
    return study

# start search

In [5]:
EXP_ID = "exp6_aug_calib"
keras.utils.disable_interactive_logging()

# start searching for a model
study = search()

# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y:%m:%d:%H:%M:%S") + ".pkl")

[I 2024-02-01 14:31:51,589] A new study created in memory with name: no-name-1c7e2b9e-b8ee-4dd4-906b-66fe48cda6cc


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t0/assets


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t0/assets
I0000 00:00:1706808714.879535 4096958 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
[I 2024-02-01 14:32:07,545] Trial 0 finished with value: 0.6800000071525574 and parameters: {'batchsize': 32, 'brightness_factor': 0.012177987073748354, 'gaussian_noise_factor': 0.12321480363136894, 'translate_factor': 0.18408030115464427, 'num_layers': 1, 'layer_0': 'Depthwise2D', 'layer_0_dwconv2d_fsize': 3, 'layer_0_dwconv2d_ssize': 15, 'activation_0': 'relu', 'output_dropout': 0.5789116351965652, 'lr_inicial': 0.0003016800832011265}. Best is trial 0 with value: 0.6800000071525574.


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t1/assets


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t1/assets
[I 2024-02-01 14:32:21,481] Trial 1 finished with value: 0.699999988079071 and parameters: {'batchsize': 64, 'brightness_factor': 0.13776367850825771, 'gaussian_noise_factor': 0.17357461542801864, 'translate_factor': 0.0917444188163728, 'num_layers': 1, 'layer_0': 'Pointwise2D', 'layer_0_pwconv2d_filters': 28, 'layer_0_pwconv2d_ssize': 12, 'activation_0': 'tanh', 'output_dropout': 0.49699128438366247, 'lr_inicial': 0.00045741691344391685}. Best is trial 1 with value: 0.699999988079071.


...

INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t298/assets


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t298/assets
[I 2024-02-01 16:07:28,526] Trial 298 finished with value: 0.8399999737739563 and parameters: {'batchsize': 128, 'brightness_factor': 0.14674797874756662, 'gaussian_noise_factor': 0.02309744622494604, 'translate_factor': 0.1997428823298305, 'num_layers': 3, 'layer_0': 'Pointwise2D', 'layer_0_pwconv2d_filters': 20, 'layer_0_pwconv2d_ssize': 2, 'activation_0': 'tanh', 'layer_1': 'Conv2D', 'layer_1_conv2d_filters': 24, 'layer_1_conv2d_fsize': 5, 'layer_1_conv2d_ssize': 2, 'activation_1': 'tanh', 'layer_2': 'Depthwise2D', 'layer_2_dwconv2d_fsize': 4, 'layer_2_dwconv2d_ssize': 4, 'activation_2': 'relu', 'output_dropout': 0.5131454503689198, 'lr_inicial': 0.0010994882619495187}. Best is trial 85 with value: 0.9800000190734863.


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t299/assets


INFO:tensorflow:Assets written to: saved_models/exp6_aug_calib/best_model_t299/assets
[I 2024-02-01 16:07:50,149] Trial 299 finished with value: 0.7400000095367432 and parameters: {'batchsize': 32, 'brightness_factor': 0.18777975375933537, 'gaussian_noise_factor': 0.010220956974527066, 'translate_factor': 0.17869918811098756, 'num_layers': 3, 'layer_0': 'Pointwise2D', 'layer_0_pwconv2d_filters': 20, 'layer_0_pwconv2d_ssize': 2, 'activation_0': 'tanh', 'layer_1': 'Conv2D', 'layer_1_conv2d_filters': 24, 'layer_1_conv2d_fsize': 5, 'layer_1_conv2d_ssize': 2, 'activation_1': 'tanh', 'layer_2': 'Pointwise2D', 'layer_2_pwconv2d_filters': 4, 'layer_2_pwconv2d_ssize': 4, 'activation_2': 'relu', 'output_dropout': 0.5327841143916707, 'lr_inicial': 0.0009711985504487472}. Best is trial 85 with value: 0.9800000190734863.


['saved_studies/2024:02:01:16:07:50.pkl']

# visualizations

In [6]:
print("Number of finished trials: ", len(study.trials))

trial = study.best_trial
print(f"Best trial id: {trial._trial_id}")

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  300
Best trial id: 85
  Value:  0.9800000190734863
  Params: 
    batchsize: 32
    brightness_factor: 0.11014907799381678
    gaussian_noise_factor: 0.00012251879039760355
    translate_factor: 0.17819308851542753
    num_layers: 3
    layer_0: Pointwise2D
    layer_0_pwconv2d_filters: 20
    layer_0_pwconv2d_ssize: 3
    activation_0: relu
    layer_1: Conv2D
    layer_1_conv2d_filters: 24
    layer_1_conv2d_fsize: 5
    layer_1_conv2d_ssize: 2
    activation_1: tanh
    layer_2: Depthwise2D
    layer_2_dwconv2d_fsize: 4
    layer_2_dwconv2d_ssize: 4
    activation_2: relu
    output_dropout: 0.5532897757283509
    lr_inicial: 0.001110531777036327


In [12]:
study_2 = joblib.load("/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_studies/2024:02:01:16:07:50.pkl")

In [13]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [14]:
plot_optimization_history(study_2)

In [15]:
plot_param_importances(study_2)

In [16]:
plot_parallel_coordinate(
    study_2,
    [
        "lr_inicial",
        "num_layers",
        "layer_0",
        "activation_0",
    ],
)